In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
model = YOLO("/content/drive/MyDrive/Endoscope Detection/Yolov11/experiment/v3/exp33/weights/best.pt")

In [6]:
meta_data = pd.read_csv('/content/drive/MyDrive/Endoscope Detection/Yolov11/gastroscopy_meta_256.csv')

In [7]:
!pip install -q map-boxes
from map_boxes import mean_average_precision_for_boxes

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
img_paths = ["/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test"]

output_data = []

def format_pred(labels, boxes, scores):
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.float32)
        pred_strings.append(f"{label} {score:.4f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}")
    return " ".join(pred_strings)

for img_path in img_paths:
    results = model(img_path, save=True)
    for result in results:
        image_id = result.path.split("/")[-1]
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        labels = result.boxes.cls.cpu().numpy().astype(int)

        pred_str = format_pred(labels, boxes, scores)
        output_data.append([image_id, pred_str])


image 1/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00021.png: 256x256 (no detections), 20.4ms
image 2/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00023.png: 256x256 1 ulcer, 9.3ms
image 3/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00025.png: 256x256 2 ulcers, 9.4ms
image 4/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00033.png: 256x256 (no detections), 9.6ms
image 5/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00035.png: 256x256 (no detections), 22.5ms
image 6/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00044.png: 256x256 1 ulcer, 20.4ms
image 7/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00045.png: 256x256 3 ulcers, 13.4ms
image 8/400 /content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/1_1_00142.png: 256x256 3 ulcers, 9.1ms


In [10]:
submission_det = pd.DataFrame(output_data, columns=['image_id','PredictionString'])

In [11]:
submission_det

,image_id,PredictionString
0,1_1_00021.png,
1,1_1_00023.png,0 0.4378 76.9 29.6 256.0 164.7
2,1_1_00025.png,0 0.4736 145.9 77.9 234.6 178.3 0 0.4632 160.3...
3,1_1_00033.png,
4,1_1_00035.png,
...,...,...
395,1_3_09783.png,2 0.8637 3.8 16.5 256.0 236.3
396,1_3_09808.png,2 0.8872 59.0 25.7 256.0 183.7
397,1_3_09826.png,2 0.8307 56.4 16.8 228.8 140.8
398,1_3_09872.png,2 0.7935 4.5 18.5 255.0 238.1 2 0.6584 0.1 159...


In [12]:
def string_decoder(predict_string):
    s = predict_string.split(' ')
    class_id = [x for x in (s[0::6])]
    score = [x for x in (s[1::6])]
    x_min = [x for x in (s[2::6])]
    y_min = [x for x in (s[3::6])]
    x_max = [x for x in (s[4::6])]
    y_max = [x for x in (s[5::6])]

    return class_id, score, x_min, y_min, x_max, y_max

In [13]:
submission_list = []

for i, row in tqdm(submission_det.iterrows(), total=len(submission_det)):
    image_id = row['image_id']
    string_decoded = string_decoder(row['PredictionString'])

    for  (class_id, score, xmin, ymin, xmax, ymax) in zip(*string_decoded):
        submission_list.append(
            {
                'image_id': image_id,
                'lesion_label': int(class_id),
                'score': float(score),
                'x_min': float(xmin),
                'y_min': float(ymin),
                'x_max': float(xmax),
                'y_max': float(ymax),
                }
        )

submission_metric = pd.DataFrame(submission_list, columns=['image_id', 'lesion_label', 'score','x_min','y_min', 'x_max', 'y_max'])

100%|██████████| 400/400 [00:00<00:00, 19642.92it/s]


In [14]:
test_id = [x.split('/')[-1] for x in glob('/content/drive/MyDrive/Endoscope Detection/Yolov11/Dataset/images/test/*.png')]
test_df = meta_data[meta_data['image_id'].isin(test_id)]
test_df = test_df.sort_values(by='image_id').reset_index(drop=True)

In [15]:
ann = test_df.copy()
det = submission_metric.copy()

ann = ann[['image_id','lesion_label','x_min','x_max','y_min','y_max']].values

det = det[['image_id','lesion_label', 'score','x_min','x_max','y_min','y_max']].values


mean_ap_50, average_precisions_50 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.5)
print("_________________________________________________________")
mean_ap_75, average_precisions_75 = mean_average_precision_for_boxes(ann, det, iou_threshold=0.75)

Number of files in annotations: 398
Number of files in predictions: 365
Unique classes: 3
Detections length: 365
Annotations length: 398
0                              | 0.390059 |     148
1                              | 0.548369 |     270
2                              | 0.658937 |     231
mAP: 0.532455
_________________________________________________________
Number of files in annotations: 398
Number of files in predictions: 365
Unique classes: 3
Detections length: 365
Annotations length: 398
0                              | 0.180967 |     148
1                              | 0.309718 |     270
2                              | 0.412485 |     231
mAP: 0.301057
